In [ ]:
"""
Experiments on semi-synthetic dataset with mask
"""
from MF import MF
import pickle
import numpy as np
import torch

def getAccuracy(real, predList):
    returnList = []
    for i in predList:
        returnList.append(np.abs(real-i)/real)
    return np.array(returnList)

def calculate_g_ml_100k (x_train_user, x_train_item, num_user = 943, num_item = 1682, gamma = 353):
    interference_matrix = np.zeros((num_user, num_item))
    x = np.zeros((num_user, num_item))
    for i in range(len(x_train_user)):
            x[x_train_user[i],x_train_item[i]] = 1
    for i in range(num_user):
        interference_matrix[i,:] += np.sum(x[i,:])# + np.sum(x[:,j])
    for j in range(num_item):
        interference_matrix[:,j] += np.sum(x[:,j])
    
    interference_matrix -= x
    
    inter_matrix = np.ones((943,1682))
    
    for i in range(num_user):
        m = interference_matrix[i,:] >= gamma
        inter_matrix[i,:] *= m        
    return inter_matrix

file = open("data/synthetic_data", "rb")
ground_truth = pickle.load(file)
one = pickle.load(file)
three = pickle.load(file)
four = pickle.load(file)
rotate = pickle.load(file)
skew = pickle.load(file)
crs = pickle.load(file)
a = pickle.load(file)
file.close()


#then compute g

matrix = np.loadtxt("./data/u.data", dtype=int)
user = matrix[:, 0] - 1
item = matrix[:, 1] - 1
rating = matrix[:, 2]
user_num = np.max(user)+1
item_num = np.max(item)+1
num_user = 943
num_item = 1682

matrix_g = calculate_g_ml_100k(user, item, gamma = 353)

for i in range(len(user)):
    if matrix_g[user[i], item[i]] == 1:
         matrix[i, 3] = 1
    else:
         matrix[i, 3] = 0

user_g1 = []        
item_g1 = []        
rating_g1 = []

for i in range(len(user)):
    if matrix[i, 3] == 1:
        user_g1.append(user[i])
        item_g1.append(item[i])
        rating_g1.append(rating[i])

total_num = len(user_g1)        

user_g1 = np.array(user_g1)
item_g1 = np.array(item_g1)
rating_g1 = np.array(rating_g1)

user_train, item_train, rating_train = user_g1[:int(total_num*0.9)], item_g1[:int(total_num*0.9)], rating_g1[:int(total_num*0.9)]
user_test, item_test, rating_test = user_g1[int(total_num*0.9):], item_g1[int(total_num*0.9):], rating_g1[int(total_num*0.9):]       

mf = MF(num_users=user_num, num_items=item_num, embedding_size=64)
mf.fit(user_train, item_train, rating_train, user_test, item_test, rating_test, batch_size = 256, lamb=1e-3, gamma = 1e-4)  

all_matrix = np.array([[x0, y0] for x0 in np.arange(user_num) for y0 in np.arange(item_num)]) # all user item pair
user_all = all_matrix[:, 0] #
item_all = all_matrix[:, 1] # 
#print(user_all)
#print(item_all)
rating_all = np.zeros(user_all.shape)
prediction_g1 = mf.predict(user_all, item_all)

total_num_g1 = prediction_g1.shape[0]
index = np.argsort(prediction_g1)
index_inverse = np.argsort(index)
prediction = prediction_g1[index]
prediction[np.where(prediction <= a[0])] = 1 
prediction[np.intersect1d(np.where(prediction > a[0]), np.where(prediction <= a[1]))] = 2
prediction[np.intersect1d(np.where(prediction > a[1]), np.where(prediction <= a[2]))] = 3
prediction[np.intersect1d(np.where(prediction > a[2]), np.where(prediction <= a[3]))] = 4
prediction[np.where(prediction > a[3])] = 5
ground_truth_g1= prediction[index_inverse]
print(ground_truth_g1[:50])


user_g0 = []        
item_g0 = []        
rating_g0 = []

for i in range(len(user)):
    if matrix[i, 3] == 0:
        user_g0.append(user[i])
        item_g0.append(item[i])
        rating_g0.append(rating[i])

total_num = len(user_g0)          

user_g0 = np.array(user_g0)
item_g0 = np.array(item_g0)
rating_g0 = np.array(rating_g0)

user_train, item_train, rating_train = user_g0[:int(total_num*0.9)], item_g0[:int(total_num*0.9)], rating_g0[:int(total_num*0.9)]
user_test, item_test, rating_test = user_g0[int(total_num*0.9):], item_g0[int(total_num*0.9):], rating_g0[int(total_num*0.9):]       

mf = MF(num_users=user_num, num_items=item_num, embedding_size=64)
mf.fit(user_train, item_train, rating_train, user_test, item_test, rating_test, batch_size = 256, lamb=1e-3, gamma = 1e-4)  

prediction_g0 = mf.predict(user_all, item_all)

total_num_g0 = prediction_g0.shape[0]
index = np.argsort(prediction_g0)
index_inverse = np.argsort(index)
prediction = prediction_g0[index]
prediction[np.where(prediction <= a[0])] = 1 
prediction[np.intersect1d(np.where(prediction > a[0]), np.where(prediction <= a[1]))] = 2
prediction[np.intersect1d(np.where(prediction > a[1]), np.where(prediction <= a[2]))] = 3
prediction[np.intersect1d(np.where(prediction > a[2]), np.where(prediction <= a[3]))] = 4
prediction[np.where(prediction > a[3])] = 5
ground_truth_g0= prediction[index_inverse] 
print(ground_truth_g0[:50])

# ratio = [0.53, 0.24, 0.14, 0.06, 0.03]
# predList = [one, three, four, rotate, skew, crs]
# store the experiment results with different mask ratios
one_means_dr = []
one_stds_dr = []
one_means_inter_dr = []
one_stds_inter_dr = []
one_means_mrdr = []
one_stds_mrdr = []
one_means_inter_mrdr = []
one_stds_inter_mrdr = []

three_means_dr = []
three_stds_dr = []
three_means_inter_dr = []
three_stds_inter_dr = []
three_means_mrdr = []
three_stds_mrdr = []
three_means_inter_mrdr = []
three_stds_inter_mrdr = []

four_means_dr = []
four_stds_dr = []
four_means_inter_dr = []
four_stds_inter_dr = []
four_means_mrdr = []
four_stds_mrdr = []
four_means_inter_mrdr = []
four_stds_inter_mrdr = []

rotate_means_dr = []
rotate_stds_dr = []
rotate_means_inter_dr = []
rotate_stds_inter_dr = []
rotate_means_mrdr = []
rotate_stds_mrdr = []
rotate_means_inter_mrdr = []
rotate_stds_inter_mrdr = []

skew_means_dr = []
skew_stds_dr = []
skew_means_inter_dr = []
skew_stds_inter_dr = []
skew_means_mrdr = []
skew_stds_mrdr = []
skew_means_inter_mrdr = []
skew_stds_inter_mrdr = []

crs_means_dr = []
crs_stds_dr = []
crs_means_inter_dr = []
crs_stds_inter_dr = []
crs_means_mrdr = []
crs_stds_mrdr = []
crs_means_inter_mrdr = []
crs_stds_inter_mrdr = []


for k in [50,150,250,350]:
    propensity = np.copy(ground_truth)
    mask_user_ratio = np.array([(943 - k)/943] * 943)
    mask_user = np.random.binomial(1, mask_user_ratio)
    mask_item_ratio = np.array([(1682 - k)/1682] * 1682)
    mask_item = np.random.binomial(1, mask_item_ratio)

    mask_matrix = np.outer(mask_user, mask_item)
    print(mask_matrix.shape)

    alpha = 0.5

    mask_user_number = np.sum(mask_user)
    mask_item_number = np.sum(mask_item)
    rest_number = 943*1682 - 943* (1682-mask_item_number) - 1682 * (943-mask_user_number) + (943-mask_user_number) * (1682 - mask_item_number)

    p = (0.05 * 943 * 1682 / rest_number)/(0.53*(alpha ** 3) + 0.24 * (alpha ** 2) + 0.14 * alpha + 0.09)
    print(p)
    propensity[np.where(ground_truth == 5)] = p
    propensity[np.where(ground_truth == 4)] = p 
    propensity[np.where(ground_truth == 3)] = p * alpha
    propensity[np.where(ground_truth == 2)] = p * (alpha ** 2)
    propensity[np.where(ground_truth == 1)] = p * (alpha ** 3)
    res = np.zeros([6, 4])
    res_var = np.zeros([6, 4])
    propensity = propensity.reshape(943,1682)
    propensity = mask_matrix * propensity
    propensity = propensity.reshape(943*1682)
    count = 0
    for i in range(50):
        print()
        observation = np.random.binomial(1, propensity)

        ones = np.count_nonzero(observation)
        zeros = observation.shape[0] - ones
        p_o = ones/(ones+zeros)
        #print(p_o)
        o = np.where(observation == 1)
    
        observation = observation.reshape(943, 1682)
    
        interference_matrix = np.zeros((num_user, num_item))
        for i in range(num_user):
            interference_matrix[i,:] += np.sum(observation[i,:])# + np.sum(x[:,j])
        for j in range(num_item):
            interference_matrix[:,j] += np.sum(observation[:,j])
    
        interference_matrix -= observation
    
        inter_matrix = np.ones((943,1682))
    
        for i in range(num_user):
            m = interference_matrix[i,:] >= 115
           # print(m)
           # print()
            inter_matrix[i,:] *= m        
    
        observation = observation.reshape(943 * 1682)
        matrix_g_reshape = inter_matrix.reshape(943 * 1682)
    
        g1 = np.where(matrix_g_reshape == 1)
        g0 = np.where(matrix_g_reshape == 0)
    
        p_g1 = np.sum(observation * matrix_g_reshape)/ (ones+zeros)
        p_g0 = np.sum(observation * (1-matrix_g_reshape))/ (ones+zeros)

        g_index = matrix_g_reshape[o]

        ground_truth_g1_correspond = ground_truth_g1 * matrix_g_reshape
        ground_truth_g0_correspond = ground_truth_g0 * (1-matrix_g_reshape)
        ground_truth_g_full = ground_truth_g1_correspond + ground_truth_g0_correspond 

        ratio_5 = np.sum(g_index[np.where(ground_truth[o] == 5)])/len(np.where(ground_truth[o] == 5)[0]) # 98%
        ratio_4 = np.sum(g_index[np.where(ground_truth[o] == 4)])/len(np.where(ground_truth[o] == 4)[0]) # 98%
        ratio_3 = np.sum(g_index[np.where(ground_truth[o] == 3)])/len(np.where(ground_truth[o] == 3)[0]) # 94%
        ratio_2 = np.sum(g_index[np.where(ground_truth[o] == 2)])/len(np.where(ground_truth[o] == 2)[0]) # 60%
        ratio_1 = np.sum(g_index[np.where(ground_truth[o] == 1)])/len(np.where(ground_truth[o] == 1)[0]) # 25%
        
        if ratio_5 == 1 or ratio_4 == 1 or ratio_3 == 1 or ratio_2 == 1 or ratio_1 == 1:
            continue
        
        count += 1
        
        propensity_new = np.copy(propensity)
        propensity_new[np.intersect1d(np.where(ground_truth == 5), g1)] = p * ratio_5
        propensity_new[np.intersect1d(np.where(ground_truth == 5), g0)] = p * (1-ratio_5)
        propensity_new[np.intersect1d(np.where(ground_truth == 4), g1)] = p * ratio_4
        propensity_new[np.intersect1d(np.where(ground_truth == 4), g0)] = p * (1-ratio_4)
        propensity_new[np.intersect1d(np.where(ground_truth == 3), g1)] = p * alpha * ratio_3
        propensity_new[np.intersect1d(np.where(ground_truth == 3), g0)] = p * alpha * (1-ratio_3)
        propensity_new[np.intersect1d(np.where(ground_truth == 2), g1)] = p * (alpha ** 2) * ratio_2
        propensity_new[np.intersect1d(np.where(ground_truth == 2), g0)] = p * (alpha ** 2) * (1-ratio_2)
        propensity_new[np.intersect1d(np.where(ground_truth == 1), g1)] = p * (alpha ** 3) * ratio_1
        propensity_new[np.intersect1d(np.where(ground_truth == 1), g0)] = p * (alpha ** 3) * (1-ratio_1)
        
        
        number_obs_1_ratio = len(np.where(ground_truth[o] == 1)[0])/ len(o[0])
        number_obs_2_ratio = len(np.where(ground_truth[o] == 2)[0])/ len(o[0])
        number_obs_3_ratio = len(np.where(ground_truth[o] == 3)[0])/ len(o[0])
        number_obs_4_ratio = len(np.where(ground_truth[o] == 4)[0])/ len(o[0])
        number_obs_5_ratio = len(np.where(ground_truth[o] == 5)[0])/ len(o[0])

        difference_matrix = ground_truth_g1 == ground_truth_g0

        propensity_copy = propensity.copy()
        propensity_copy[np.where(propensity == 0)] = p_o
        p_hat = 0.5/propensity_copy + 0.5/p_o
        p_hat_g = np.zeros(len(propensity_new))
        p_hat_g[g1] = 0.5/propensity_new[g1] + 0.5/p_g1
        p_hat_g[g0] = 0.5/propensity_new[g0] + 0.5/p_g0
        predList = [one, three, four, rotate, skew, crs]

        c = 0.5
        for j in range(6):
            prediction = predList[j]
            ce = c * abs(ground_truth_g1 - prediction) + (1 - c) * abs(ground_truth_g0 - prediction)
        
            # DR
            prediction_hat = np.sum(prediction * p_hat * observation) / np.sum(observation * p_hat) # tilder
            ce_hat = abs(prediction_hat- prediction)
        
            # DR Inter
            prediction_hat1 = np.sum(prediction * matrix_g_reshape * p_hat_g * observation) / np.sum(observation * matrix_g_reshape * p_hat_g)
            ce_hat1 = abs(prediction_hat1- prediction)

            prediction_hat0 = np.sum(prediction * (1 - matrix_g_reshape) * p_hat_g * observation) / np.sum(
                observation * (1 - matrix_g_reshape) * p_hat_g)
        
            ce_hat0 = abs(prediction_hat0- prediction)

        
            # MRDR
            prediction_hat = np.sum(prediction * p_hat * p_hat * (1 - 1 / p_hat) * observation) / np.sum(
                p_hat * p_hat * (1 - 1 / p_hat) * observation)

            ce_mrdr = abs(prediction_hat- prediction) # MRDR e_hat
        
            # MRDR Inter
        
            prediction_hat1 = np.sum(prediction * matrix_g_reshape * p_hat_g * p_hat_g * (1 - 1 / p_hat_g) * observation) / np.sum(
                observation * matrix_g_reshape * p_hat_g * p_hat_g * (1 - 1 / p_hat_g))
        
            ce_mrdr1 = abs(prediction_hat1- prediction)
            
            prediction_hat0 = np.sum(prediction * (1 - matrix_g_reshape) * p_hat_g * p_hat_g * (1 - 1 / p_hat_g) * observation) / np.sum(
                observation * (1 - matrix_g_reshape) * p_hat_g * p_hat_g * (1 - 1 / p_hat_g))
        
            ce_mrdr0 = abs(prediction_hat0- prediction)
        
            real_ce = np.mean(ce)
            naive_ce = np.sum(abs(ground_truth_g1 - prediction)* observation * matrix_g_reshape + 
                             abs(ground_truth_g0 - prediction) * observation * (1-matrix_g_reshape))/np.sum(observation)

            ips_ce = np.mean(abs(ground_truth_g1 - prediction) * observation * matrix_g_reshape * p_hat + 
                             abs(ground_truth_g0 - prediction) * observation * (1-matrix_g_reshape) * p_hat)
        
            ips_ce_with_g = np.mean(c * abs(ground_truth_g1 - prediction) * observation * matrix_g_reshape * p_hat_g + 
                             (1-c) * abs(ground_truth_g0 - prediction) * (1-matrix_g_reshape) * observation * p_hat_g)
        
            dr_ce = np.mean(ce_hat + observation * matrix_g_reshape * (abs(ground_truth_g1 - prediction) - ce_hat) * p_hat +
                           observation * (1 - matrix_g_reshape) * (abs(ground_truth_g0 - prediction) - ce_hat) * p_hat)
        
            dr_ce_with_g = np.mean(c*(ce_hat1 + observation * matrix_g_reshape * (abs(ground_truth_g1 - prediction) - ce_hat1) * p_hat_g) +
                                  (1-c)*(ce_hat0 + observation * (1-matrix_g_reshape) * (abs(ground_truth_g0 - prediction) - ce_hat0) * p_hat_g))
                                      
            mrdr_ce = np.mean(ce_mrdr + observation * matrix_g_reshape * (abs(ground_truth_g1 - prediction) - ce_mrdr) * p_hat +
                           observation * (1 - matrix_g_reshape) * (abs(ground_truth_g0 - prediction) - ce_mrdr) * p_hat)
        
            mrdr_ce_with_g = np.mean(c*(ce_mrdr1 + observation * matrix_g_reshape * (abs(ground_truth_g1 - prediction) - ce_mrdr1) * p_hat_g) +
                                  (1-c)*(ce_mrdr0 + observation * (1-matrix_g_reshape) * (abs(ground_truth_g0 - prediction) - ce_mrdr0) * p_hat_g))
            acc = getAccuracy(real_ce, [dr_ce, dr_ce_with_g, mrdr_ce, mrdr_ce_with_g])
            res[j] += acc
            res_var[j] += acc ** 2
            
            print(acc)

    print()
    loss_le = res/count
    print(loss_le)
    loss_std = np.sqrt((1/(count-1))*(res_var - count*(res/count)**2))
    print(loss_std)
    
    # store the results
    one_means_dr.append(loss_le[0][0])
    one_stds_dr.append(loss_std[0][0])
    one_means_inter_dr.append(loss_le[0][1])
    one_stds_inter_dr.append(loss_std[0][1])
    one_means_mrdr.append(loss_le[0][2])
    one_stds_mrdr.append(loss_std[0][2])
    one_means_inter_mrdr.append(loss_le[0][3])
    one_stds_inter_mrdr.append(loss_std[0][3])

    three_means_dr.append(loss_le[1][0])
    three_stds_dr.append(loss_std[1][0])
    three_means_inter_dr.append(loss_le[1][1])
    three_stds_inter_dr.append(loss_std[1][1])
    three_means_mrdr.append(loss_le[1][2])
    three_stds_mrdr.append(loss_std[1][2])
    three_means_inter_mrdr.append(loss_le[1][3])
    three_stds_inter_mrdr.append(loss_std[1][3])

    four_means_dr.append(loss_le[2][0])
    four_stds_dr.append(loss_std[2][0])
    four_means_inter_dr.append(loss_le[2][1])
    four_stds_inter_dr.append(loss_std[2][1])
    four_means_mrdr.append(loss_le[2][2])
    four_stds_mrdr.append(loss_std[2][2])
    four_means_inter_mrdr.append(loss_le[2][3])
    four_stds_inter_mrdr.append(loss_std[2][3])

    rotate_means_dr.append(loss_le[3][0])
    rotate_stds_dr.append(loss_std[3][0])
    rotate_means_inter_dr.append(loss_le[3][1])
    rotate_stds_inter_dr.append(loss_std[3][1])
    rotate_means_mrdr.append(loss_le[3][2])
    rotate_stds_mrdr.append(loss_std[3][2])
    rotate_means_inter_mrdr.append(loss_le[3][3])
    rotate_stds_inter_mrdr.append(loss_std[3][3])

    skew_means_dr.append(loss_le[4][0])
    skew_stds_dr.append(loss_std[4][0])
    skew_means_inter_dr.append(loss_le[4][1])
    skew_stds_inter_dr.append(loss_std[4][1])
    skew_means_mrdr.append(loss_le[4][2])
    skew_stds_mrdr.append(loss_std[4][2])
    skew_means_inter_mrdr.append(loss_le[4][3])
    skew_stds_inter_mrdr.append(loss_std[4][3])

    crs_means_dr.append(loss_le[5][0])
    crs_stds_dr.append(loss_std[5][0])
    crs_means_inter_dr.append(loss_le[5][1])
    crs_stds_inter_dr.append(loss_std[5][1])
    crs_means_mrdr.append(loss_le[5][2])
    crs_stds_mrdr.append(loss_std[5][2])
    crs_means_inter_mrdr.append(loss_le[5][3])
    crs_stds_inter_mrdr.append(loss_std[5][3])